In [1]:
import pandas as pd
import numpy as np
import os
import json
import random

In [2]:
#Variables de los ficheros de datos salida
CurrentAccountKeyspace_file_out = '../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountKeyspace.csv'
CurrentAccountKeyspaceAccountInfo_file_out = '../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountAccountInfoKeyspace.csv'
PositionKeepingKeyspace_file_out = '../MockData/Cassandra/PositionKeepingKeyspace/PositionKeepingKeyspace.csv'
CustomerProfileKeyspace_file_out = '../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileKeyspace.csv'
CustomerProfileAddressKeyspace_file_out = '../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileAddressKeyspace.csv'

CurrentAccountKeyspace_sample_out = '../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountKeyspace_sample.csv'
CurrentAccountKeyspaceAccountInfo_sample_out = '../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountAccountInfoKeyspace_sample.csv'
PositionKeepingKeyspace_sample_out = '../MockData/Cassandra/PositionKeepingKeyspace/PositionKeepingKeyspace_sample.csv'
CustomerProfileKeyspaceCustomer_sample_out = '../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileKeyspace_sample.csv'
CustomerProfileKeyspaceAddress_sample_out = '../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileAddressKeyspace_sample.csv'

In [4]:
#Función para resetear ficheros de salida
def reset_files(file):
    if os.path.exists(file):
        os.remove(file)
        print("The file", file ,"have been removed")
    else:
        print("The file", file ,"does not exist")

In [5]:
#Limpiamos los ficheros de salida
reset_files(CurrentAccountKeyspace_file_out)
reset_files(CurrentAccountKeyspaceAccountInfo_file_out)
reset_files(PositionKeepingKeyspace_file_out)
reset_files(CustomerProfileKeyspace_file_out)
reset_files(CustomerProfileAddressKeyspace_file_out)

reset_files(CurrentAccountKeyspace_sample_out)
reset_files(CurrentAccountKeyspaceAccountInfo_sample_out)
reset_files(PositionKeepingKeyspace_sample_out)
reset_files(CustomerProfileKeyspaceCustomer_sample_out)
reset_files(CustomerProfileKeyspaceAddress_sample_out)

The file ../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountKeyspace.csv does not exist
The file ../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountAccountInfoKeyspace.csv does not exist
The file ../MockData/Cassandra/PositionKeepingKeyspace/PositionKeepingKeyspace.csv does not exist
The file ../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileKeyspace.csv does not exist
The file ../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileAddressKeyspace.csv does not exist
The file ../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountKeyspace_sample.csv have been removed
The file ../MockData/Cassandra/CurrentAccountKeyspace/CurrentAccountAccountInfoKeyspace_sample.csv have been removed
The file ../MockData/Cassandra/PositionKeepingKeyspace/PositionKeepingKeyspace_sample.csv have been removed
The file ../MockData/Cassandra/CustomerProfileKeyspace/CustomerProfileKeyspace_sample.csv have been removed
The file ../MockData/Cassandra/CustomerProfileKeyspace/Cu

# Generacion ficheros keyspace CustomerProfile

In [6]:
#Carga de la información de dataframes por entidades

Address_df = pd.read_csv('../MockData/Address_1M.csv')
CustomerProfile_df = pd.read_csv('../MockData/CustomerProfile_1M.csv')
Country_df = pd.read_csv('../MockData/base/Country.csv')

In [7]:
# --- NO EJECUTAR ESTA LINEA SI SE GENERA EN MODO REAL ---- #
CustomerProfile_df = CustomerProfile_df.sample(1000) #Comentar si modo real
Address_df = Address_df.sample(1000)

In [8]:
CustomerProfile_df.sample(1)

,PartyId,PartyNumber,PartyType,Name,FullLegalName,LegalStructure,BeneficialOwnership,AccountRole,EmailAddress,Phone
1275728,2327,2327,sol,Jovany,Demario Rau PhD,Ms.,3,UK.OBIE.Secundary,kjones@example.com,(517)315-684


In [9]:
Address_df.sample(1)

,PartyId,AddressType,AddressLine,StreetName,BuildingNumber,PostCode,TownName,CountrySubDivision,CountryId
974571,974571,Port,077 Stiedemann Land Suite 120,Jaquan Vista,6185,03465-0381,New Taureanbury,28.0,3


In [10]:
Country_df.sample(1)

,CountryId,ShortName,Description,Code,Unnamed: 4
2,3,Greatbrit,Aut sapiente dicta illo consectetur nulla labo...,SPA,NaN


In [11]:
#Desnormalizamos la información para la carga en Cassandra
Country_Code_column = []
Country_ShortName_column = []
Country_Description_column = []

for i in range (0,len(Address_df)): 
    country_deployed_df = Country_df.sample(1)
    Country_Code_column.append(country_deployed_df['Code'].iloc[0])
    Country_ShortName_column.append(country_deployed_df['ShortName'].iloc[0])
    Country_Description_column.append(country_deployed_df['Description'].iloc[0])

Address_df['Country_Code'] = Country_Code_column
Address_df['Country_ShortName'] = Country_ShortName_column
Address_df['Country_Description'] = Country_Description_column

Address_df.sample(2)

,PartyId,AddressType,AddressLine,StreetName,BuildingNumber,PostCode,TownName,CountrySubDivision,CountryId,Country_Code,Country_ShortName,Country_Description
731170,731170,Lake,507 Howe Knoll,Gregorio Unions,1928,36318-0327,Omerfurt,10.0,1,IND,Spain,Omnis dolore id dicta consequatur sit omnis. E...
98205,98205,Nort,4060 Pfeffer Isle,Elwyn Plaza,6948,58227-1783,Gracemouth,619494.0,3,GBP,USA,Id dolorem a soluta harum iusto qui repellat. ...


In [12]:
#Volcado de los ficheros relacionados con Customer Profile (la carga de direcciones se realizará en CQL)
#Samples (COMENTAR EN MODO FINAL)
CustomerProfile_df.to_csv(CustomerProfileKeyspaceCustomer_sample_out, index= False)
Address_df.to_csv(CustomerProfileKeyspaceAddress_sample_out, index= False)

#Modo final
#CustomerProfile_df.to_csv(CustomerProfileKeyspace_file_out, index= False)
#Address_df.to_csv(CustomerProfileAddressKeyspace_file_out, index= False)

# Generacion de ficheros KeySpace CurrentAccount

In [13]:
#Carga de la información de dataframes por entidades

AccountInfo_df = pd.read_csv('../MockData/AccountInfo_1M.csv')
CurrentAccount_df = pd.read_csv('../MockData/CurrentAccount_1M.csv')

In [14]:
# --- NO EJECUTAR ESTA LINEA SI SE GENERA EN MODO REAL ---- #
CurrentAccount_df = CurrentAccount_df.sample(1000)
AccountInfo_df = AccountInfo_df.sample(250)

In [15]:
CurrentAccount_df.sample(1)

,AccountId,PartyId,Status,StatusUpdateDateTime,AccountType,NickName,OpeningDate,AccountSubType
1132588,7119,2888,Enabled,2010-05-20 00:31:13.00,Business,deliver B2B e-services,1996-03-31 13:39:03.00,POI


In [16]:
AccountInfo_df.sample(1)

,AccountInfoId,AccountId,SchemeName,Identification,Name
369320,8748,8749,NaN,9.0,"Hansen, Ondricka and Rowe"


In [17]:
#Limpieza de columnas no necesarias en el modelo columnar
del(AccountInfo_df['AccountInfoId'])

In [18]:
CurrentAccount_df.sample(1)

,AccountId,PartyId,Status,StatusUpdateDateTime,AccountType,NickName,OpeningDate,AccountSubType
691222,4924,9170,Enabled,1988-12-31 14:31:41.00,Business,orchestrate leading-edge technologies,2005-07-28 14:04:53.00,SAI


In [19]:
AccountInfo_df.sample(1)

,AccountId,SchemeName,Identification,Name
965376,7937,NaN,8.0,Moore LLC


In [20]:
#Volcado de los ficheros relacionados con Current Account (la carga de direcciones se realizará en CQL)
#Samples (COMENTAR EN MODO FINAL)
CurrentAccount_df.to_csv(CurrentAccountKeyspace_sample_out, index= False)
AccountInfo_df.to_csv(CurrentAccountKeyspaceAccountInfo_sample_out, index= False)

#Modo final
#CurrentAccount_df.to_csv(CurrentAccountKeyspace_file_out, index= False)
#AccountInfo_df.to_csv(CurrentAccountKeyspaceAccountInfo_file_out, index= False)

# Generacion de ficheros KeySpace PositionKeeping


In [21]:
#Carga de datasets necesarias para la colección
Amount_df = pd.read_csv('../MockData/Amount_1M.csv')
CreditLine_df = pd.read_csv('../MockData/CreditLine_1M.csv')
Currency_df = pd.read_csv('../MockData/base/Currency.csv')
PositionKeeping_df = pd.read_csv('../MockData/Position_Keeping_1M.csv')

In [22]:
print(Amount_df.columns)
print(CreditLine_df.columns)
print(Currency_df.columns)
print(PositionKeeping_df.columns)

Index(['AmountId', 'CurrencyId', 'Amount'], dtype='object')
Index(['CreditLineId', 'CurrencyId', 'Amount', 'Type', 'Included'], dtype='object')
Index(['CurrencyId', 'Code', 'Description', 'Unnamed: 3'], dtype='object')
Index(['AccountId', 'DateTime', 'CreditDebitIndicator', 'Type', 'AmountId',
       'CreditLineId'],
      dtype='object')


In [23]:
#Limpieza de columnas residuales y no necesarias en el documento
del(Amount_df['CurrencyId'])
del(Amount_df['AmountId'])

del(CreditLine_df['CreditLineId'])
del(CreditLine_df['CurrencyId'])

del(Currency_df['CurrencyId'])

del(PositionKeeping_df['CreditLineId'])
del(PositionKeeping_df['AmountId'])

In [24]:
#Estado final de atributos para modelo columnar
print(Amount_df.columns)
print(CreditLine_df.columns)
print(Currency_df.columns)
print(PositionKeeping_df.columns)

Index(['Amount'], dtype='object')
Index(['Amount', 'Type', 'Included'], dtype='object')
Index(['Code', 'Description', 'Unnamed: 3'], dtype='object')
Index(['AccountId', 'DateTime', 'CreditDebitIndicator', 'Type'], dtype='object')


In [25]:
#Creamos samples. Testing (comentar en generación real)
Amount_df = Amount_df.sample(500)
CreditLine_df = CreditLine_df.sample(500)
PositionKeeping_df = PositionKeeping_df.sample(1000)

In [26]:
#Procesado de datos PositionKeeping para desnormalizar las entidades en una sola columna compatible con el modelo columnar
Amount_Amount_column = []
Amount_Currency_Description_column = []
Amount_Currency_Code_column = []
CreditLine_Amount_column = []
CreditLine_Currency_Description_column = []
CreditLine_Currency_Code_column = []
CreditLine_Type_column = []
CreditLine_Included_column = []

for i in range (0,len(PositionKeeping_df)):    
    Amount_Amount_column.append(Amount_df.sample(1)['Amount'].iloc[0])
    Amount_Currency_Description_column.append(Currency_df.sample(1)['Description'].iloc[0])
    Amount_Currency_Code_column.append(Currency_df.sample(1)['Code'].iloc[0])
    CreditLine_Amount_column.append(CreditLine_df.sample(1)['Amount'].iloc[0])
    CreditLine_Currency_Description_column.append(Currency_df.sample(1)['Description'].iloc[0])
    CreditLine_Currency_Code_column.append(Currency_df.sample(1)['Code'].iloc[0])
    CreditLine_Type_column.append(CreditLine_df.sample(1)['Type'].iloc[0])
    CreditLine_Included_column.append(CreditLine_df.sample(1)['Included'].iloc[0])

PositionKeeping_df['Amount_Amount'] = Amount_Amount_column
PositionKeeping_df['Amount_Currency_Description'] = Amount_Currency_Description_column
PositionKeeping_df['Amount_Currency_Code'] = Amount_Currency_Code_column
PositionKeeping_df['CreditLine_Amount']= CreditLine_Amount_column
PositionKeeping_df['CreditLine_Currency_Description']= CreditLine_Currency_Description_column
PositionKeeping_df['CreditLine_Currency_Code']= CreditLine_Currency_Code_column
PositionKeeping_df['CreditLine_Type']= CreditLine_Type_column
PositionKeeping_df['CreditLine_Included']= CreditLine_Included_column

PositionKeeping_df.sample(3)

,AccountId,DateTime,CreditDebitIndicator,Type,Amount_Amount,Amount_Currency_Description,Amount_Currency_Code,CreditLine_Amount,CreditLine_Currency_Description,CreditLine_Currency_Code,CreditLine_Type,CreditLine_Included
258243,4160,1979-03-16 16:11:55.00,Credit,Particular,918.21,Est quidem repellendus dolor qui. Eos temporib...,IND,1.737503e+08,Est quidem repellendus dolor qui. Eos temporib...,GBP,NaN,0
422709,6498,1979-12-31 06:35:54.00,Debit,Particular,0.00,Ut necessitatibus in iure cum et sint. Minima ...,USA,2.885542e+07,Ut necessitatibus in iure cum et sint. Minima ...,USA,NaN,0
829751,1104,1988-11-18 02:41:48.00,Credit,Particular,0.00,Dolores sint vero vero maiores nisi. Et quas i...,IND,0.000000e+00,Est quidem repellendus dolor qui. Eos temporib...,IND,NaN,0


In [27]:
#Volcado de los ficheros relacionados con Current Account (la carga de direcciones se realizará en CQL)
#Samples (COMENTAR EN MODO FINAL)
PositionKeeping_df.to_csv(PositionKeepingKeyspace_sample_out, index= False)
#PositionKeeping_df.to_csv(PositionKeepingKeyspace_file_out, index= False)